$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{Universidad de los Andes}$$
$$\textrm{Grupo de Física de Altas Energías: Fenomenología de Partículas}$$

$\textbf{Preliminares}$ 

Las librerías que se usan aquí son las siguientes: 

In [ ]:
import pandas as pd
import numpy as np
import os

Definamos las rutas que necesitamos para leer los archivos cutflows

In [ ]:
Path_significances = os.path.dirname(os.path.realpath('Sheets_wo_RHC_JP.ipynb'))
Path_Leptoquarks_searches = os.path.dirname(Path_significances)
Path_Selected_Events = f'{Path_Leptoquarks_searches}/03_delphes_preselection/Selected_Events' #Carpeta donde están los csv preseleccionados
Path_Tablas_XS = f'{Path_Leptoquarks_searches}/02_parton_analysis/XS_Matrix'
Path_Composition_Charts = f'{Path_Leptoquarks_searches}/03_delphes_preselection/Composition_Charts'

In [ ]:
channels = ['hadronic', 'semileptonic']
processes = ['b_b_tau_tau', 'b_tau_tau']
signals = ['LQ_LQ','Tau_LQ']

In [ ]:
datas = ['Data_5K_13_TeV']
betards = {'Betard33_minus1': '', 'Betard33_0': '_wo_RHC'}

In [ ]:
def Eficiencia(process, channel, signal):
    
    path_csv = f'{Path_Selected_Events}/Cutflow_{process}_{channel}.csv'
    cutflow = pd.read_csv(path_csv)
    
    return cutflow[signal][cutflow.shape[0]-2] #Así se extrae el valor de la penultima fila, esa es la eficiencia

In [ ]:
Compositions = {}
for betard in betards:
    betard_name = betard.replace('33', '')
    Compositions[betard_name] = {}
    
    for process in processes:
        for channel in channels:
            ruta = f'{process}_{channel}'
            Compositions[betard_name][ruta] = pd.read_excel(f'{Path_Composition_Charts}/{betard}/{ruta}.xlsx', index_col = 0)
            Compositions[betard_name][ruta].columns = [float(colum) for colum in Compositions[betard_name][ruta].columns]
            Compositions[betard_name][ruta].index = [float(fila) for fila in Compositions[betard_name][ruta].index]
        #Compositions[ruta].sort_index(level=0, ascending=False, inplace=True)

In [ ]:
Compositions

In [ ]:
Tablas_XS = {}
for betard in betards:
    betard_name = betard.replace('33', '')
    Tablas_XS[betard_name] = {}
    for data in datas:
        for signal in signals:
            ruta = signal.replace('LQ_LQ','dLQ').replace('Tau_LQ', 'sLQ')
            Tablas_XS[betard_name][ruta] = pd.read_excel(f'{Path_Tablas_XS}/{betard}/{data}_{signal}.xlsx', index_col = 0)
            Tablas_XS[betard_name][ruta].columns = [float(colum) for colum in Tablas_XS[betard_name][ruta].columns]
            Tablas_XS[betard_name][ruta].index = [float(fila) for fila in Tablas_XS[betard_name][ruta].index]

In [ ]:
Tablas_XS

In [ ]:
significances_files = ['all_combined', 'dLQ_combined', 'dLQ_hadronic', 'dLQ_semileptonic', 'sLQ_combined', 'sLQ_hadronic', 'sLQ_semileptonic']

In [ ]:
Significances = {}

In [ ]:
Significances['Betard_minus1'] = {}

for file in significances_files:
    
    ruta = f'{Path_significances}/Excel_Files/Betard_minus1/{file}'
    Significances['Betard_minus1'][file] = pd.read_excel(f'{ruta}.xlsx', index_col = 0)
    Significances['Betard_minus1'][file].columns = [float(colum) for colum in Significances['Betard_minus1'][file].columns]
    Significances['Betard_minus1'][file].index = [float(fila) for fila in Significances['Betard_minus1'][file].index]

In [ ]:
Significances

In [ ]:
Masses = ['1000', '1250', '1500', '1750', '2000', '2250', '2500']
g_Us = ['0.25', '0.5', '1', '1.5', '2.0', '2.5', '3.0']

Ahora usemos esto para construir las tablas de significancia con betard0

In [ ]:
Significances['Betard_0'] = {}

In [ ]:
file_name = 'sLQ'
channel = 'hadronic'
process = 'b_tau_tau' 

Significances['Betard_0'][f'{file_name}_{channel}'] = np.zeros([len(g_Us), len(Masses)])

for i in range(len(g_Us)):
    for j in range(len(Masses)):
        g_U = float(g_Us[i])
        M = float(Masses[j])

        Eficiencia_M_wo_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}')
        
        XS_wo_RHC = Tablas_XS['Betard_0'][f'{file_name}'][M][g_U]
        XS_w_RHC = Tablas_XS['Betard_minus1'][f'{file_name}'][M][g_U]
        
        Composition_wo_RHC = Compositions['Betard_0'][f'{process}_{channel}'][M][g_U]
        Composition_w_RHC = Compositions['Betard_minus1'][f'{process}_{channel}'][M][g_U]
        
        factor = np.sqrt((Eficiencia_M_wo_RHC/Eficiencia_M_w_RHC)*(XS_wo_RHC/XS_w_RHC)*(Composition_w_RHC/Composition_wo_RHC))        
        
        Significances['Betard_0'][f'{file_name}_{channel}'][i,j] = Significances['Betard_minus1'][f'{file_name}_{channel}'][M][g_U]*factor

Significances['Betard_0'][f'{file_name}_{channel}'] = pd.DataFrame(Significances['Betard_0'][f'{file_name}_{channel}'])
Significances['Betard_0'][f'{file_name}_{channel}'].columns = [float(M) for M in Masses]
Significances['Betard_0'][f'{file_name}_{channel}'].index = [float(fila) for fila in g_Us]

Significances['Betard_0'][f'{file_name}_{channel}'].to_excel(f'{Path_significances}/Excel_Files/Betard_0/{file_name}_{channel}.xlsx')

In [ ]:
file_name = 'sLQ'
channel = 'semileptonic'
process = 'b_tau_tau' 

Significances['Betard_0'][f'{file_name}_{channel}'] = np.zeros([len(g_Us), len(Masses)])

for i in range(len(g_Us)):
    for j in range(len(Masses)):
        g_U = float(g_Us[i])
        M = float(Masses[j])

        Eficiencia_M_wo_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}')
        
        XS_wo_RHC = Tablas_XS['Betard_0'][f'{file_name}'][M][g_U]
        XS_w_RHC = Tablas_XS['Betard_minus1'][f'{file_name}'][M][g_U]
        
        Composition_wo_RHC = Compositions['Betard_0'][f'{process}_{channel}'][M][g_U]
        Composition_w_RHC = Compositions['Betard_minus1'][f'{process}_{channel}'][M][g_U]
        
        factor = np.sqrt((Eficiencia_M_wo_RHC/Eficiencia_M_w_RHC)*(XS_wo_RHC/XS_w_RHC)*(Composition_w_RHC/Composition_wo_RHC))        
        
        Significances['Betard_0'][f'{file_name}_{channel}'][i,j] = Significances['Betard_minus1'][f'{file_name}_{channel}'][M][g_U]*factor

Significances['Betard_0'][f'{file_name}_{channel}'] = pd.DataFrame(Significances['Betard_0'][f'{file_name}_{channel}'])
Significances['Betard_0'][f'{file_name}_{channel}'].columns = [float(M) for M in Masses]
Significances['Betard_0'][f'{file_name}_{channel}'].index = [float(fila) for fila in g_Us]

Significances['Betard_0'][f'{file_name}_{channel}'].to_excel(f'{Path_significances}/Excel_Files/Betard_0/{file_name}_{channel}.xlsx')

In [ ]:
file_name = 'dLQ'
channel = 'hadronic'
process = 'b_b_tau_tau' 

Significances['Betard_0'][f'{file_name}_{channel}'] = np.zeros([len(g_Us), len(Masses)])

for i in range(len(g_Us)):
    for j in range(len(Masses)):
        g_U = float(g_Us[i])
        M = float(Masses[j])

        Eficiencia_M_wo_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}')
        
        XS_wo_RHC = Tablas_XS['Betard_0']['sLQ'][M][g_U]
        XS_w_RHC = Tablas_XS['Betard_minus1']['sLQ'][M][g_U]
        
        Composition_wo_RHC = Compositions['Betard_0'][f'{process}_{channel}'][M][g_U]
        Composition_w_RHC = Compositions['Betard_minus1'][f'{process}_{channel}'][M][g_U]
        
        factor = np.sqrt((Eficiencia_M_wo_RHC/Eficiencia_M_w_RHC)*(XS_wo_RHC/XS_w_RHC)*(Composition_w_RHC/Composition_wo_RHC))        
        
        Significances['Betard_0'][f'{file_name}_{channel}'][i,j] = Significances['Betard_minus1'][f'{file_name}_{channel}'][M][g_U]*factor

Significances['Betard_0'][f'{file_name}_{channel}'] = pd.DataFrame(Significances['Betard_0'][f'{file_name}_{channel}'])
Significances['Betard_0'][f'{file_name}_{channel}'].columns = [float(M) for M in Masses]
Significances['Betard_0'][f'{file_name}_{channel}'].index = [float(fila) for fila in g_Us]

Significances['Betard_0'][f'{file_name}_{channel}'].to_excel(f'{Path_significances}/Excel_Files/Betard_0/{file_name}_{channel}.xlsx')

In [ ]:
file_name = 'dLQ'
channel = 'semileptonic'
process = 'b_b_tau_tau' 

Significances['Betard_0'][f'{file_name}_{channel}'] = np.zeros([len(g_Us), len(Masses)])

for i in range(len(g_Us)):
    for j in range(len(Masses)):
        g_U = float(g_Us[i])
        M = float(Masses[j])

        Eficiencia_M_wo_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC = Eficiencia(process, channel, f'Tau_LQ_{Masses[j]}')
        
        XS_wo_RHC = Tablas_XS['Betard_0']['sLQ'][M][g_U]
        XS_w_RHC = Tablas_XS['Betard_minus1']['sLQ'][M][g_U]
        
        Composition_wo_RHC = Compositions['Betard_0'][f'{process}_{channel}'][M][g_U]
        Composition_w_RHC = Compositions['Betard_minus1'][f'{process}_{channel}'][M][g_U]
        
        factor = np.sqrt((Eficiencia_M_wo_RHC/Eficiencia_M_w_RHC)*(XS_wo_RHC/XS_w_RHC)*(Composition_w_RHC/Composition_wo_RHC))        
        
        Significances['Betard_0'][f'{file_name}_{channel}'][i,j] = Significances['Betard_minus1'][f'{file_name}_{channel}'][M][g_U]*factor

Significances['Betard_0'][f'{file_name}_{channel}'] = pd.DataFrame(Significances['Betard_0'][f'{file_name}_{channel}'])
Significances['Betard_0'][f'{file_name}_{channel}'].columns = [float(M) for M in Masses]
Significances['Betard_0'][f'{file_name}_{channel}'].index = [float(fila) for fila in g_Us]

Significances['Betard_0'][f'{file_name}_{channel}'].to_excel(f'{Path_significances}/Excel_Files/Betard_0/{file_name}_{channel}.xlsx')

Solo falta las Combinadas de cada una y All Combined.

In [ ]:
file_name = 'sLQ'
channel = 'combined'

channel_1 = 'hadronic'
channel_2 = 'semileptonic'

process = 'b_tau_tau' 

Significances['Betard_0'][f'{file_name}_{channel}'] = np.zeros([len(g_Us), len(Masses)])

for i in range(len(g_Us)):
    for j in range(len(Masses)):
        g_U = float(g_Us[i])
        M = float(Masses[j])

        Eficiencia_M_wo_RHC_1 = Eficiencia(process, channel_1, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_1 = Eficiencia(process, channel_1, f'Tau_LQ_{Masses[j]}')
        
        Eficiencia_M_wo_RHC_2 = Eficiencia(process, channel_2, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_2 = Eficiencia(process, channel_2, f'Tau_LQ_{Masses[j]}')
        
        Composition_wo_RHC_1 = Compositions['Betard_0'][f'{process}_{channel_1}'][M][g_U]
        Composition_w_RHC_1 = Compositions['Betard_minus1'][f'{process}_{channel_1}'][M][g_U]
        
        Composition_wo_RHC_2 = Compositions['Betard_0'][f'{process}_{channel_2}'][M][g_U]
        Composition_w_RHC_2 = Compositions['Betard_minus1'][f'{process}_{channel_2}'][M][g_U]
        
        XS_wo_RHC = Tablas_XS['Betard_0'][f'{file_name}'][M][g_U]
        XS_w_RHC = Tablas_XS['Betard_minus1'][f'{file_name}'][M][g_U]
        
        numerador = (Eficiencia_M_wo_RHC_1/Composition_wo_RHC_1 + Eficiencia_M_wo_RHC_2/Composition_wo_RHC_2)*XS_wo_RHC
        denominador = (Eficiencia_M_w_RHC_1/Composition_w_RHC_1 + Eficiencia_M_w_RHC_2/Composition_w_RHC_2)*XS_w_RHC
        
        factor = np.sqrt(numerador/denominador)        
        
        Significances['Betard_0'][f'{file_name}_{channel}'][i,j] = Significances['Betard_minus1'][f'{file_name}_{channel}'][M][g_U]*factor

Significances['Betard_0'][f'{file_name}_{channel}'] = pd.DataFrame(Significances['Betard_0'][f'{file_name}_{channel}'])
Significances['Betard_0'][f'{file_name}_{channel}'].columns = [float(M) for M in Masses]
Significances['Betard_0'][f'{file_name}_{channel}'].index = [float(fila) for fila in g_Us]

Significances['Betard_0'][f'{file_name}_{channel}'].to_excel(f'{Path_significances}/Excel_Files/Betard_0/{file_name}_{channel}.xlsx')

In [ ]:
file_name = 'dLQ'
channel = 'combined'

channel_1 = 'hadronic'
channel_2 = 'semileptonic'

process = 'b_b_tau_tau' 

Significances['Betard_0'][f'{file_name}_{channel}'] = np.zeros([len(g_Us), len(Masses)])

for i in range(len(g_Us)):
    for j in range(len(Masses)):
        g_U = float(g_Us[i])
        M = float(Masses[j])

        Eficiencia_M_wo_RHC_1 = Eficiencia(process, channel_1, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_1 = Eficiencia(process, channel_1, f'Tau_LQ_{Masses[j]}')
        
        Eficiencia_M_wo_RHC_2 = Eficiencia(process, channel_2, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_2 = Eficiencia(process, channel_2, f'Tau_LQ_{Masses[j]}')
        
        Composition_wo_RHC_1 = Compositions['Betard_0'][f'{process}_{channel_1}'][M][g_U]
        Composition_w_RHC_1 = Compositions['Betard_minus1'][f'{process}_{channel_1}'][M][g_U]
        
        Composition_wo_RHC_2 = Compositions['Betard_0'][f'{process}_{channel_2}'][M][g_U]
        Composition_w_RHC_2 = Compositions['Betard_minus1'][f'{process}_{channel_2}'][M][g_U]
        
        XS_wo_RHC = Tablas_XS['Betard_0']['sLQ'][M][g_U]
        XS_w_RHC = Tablas_XS['Betard_minus1']['sLQ'][M][g_U]
        
        numerador = (Eficiencia_M_wo_RHC_1/Composition_wo_RHC_1 + Eficiencia_M_wo_RHC_2/Composition_wo_RHC_2)*XS_wo_RHC
        denominador = (Eficiencia_M_w_RHC_1/Composition_w_RHC_1 + Eficiencia_M_w_RHC_2/Composition_w_RHC_2)*XS_w_RHC
        
        factor = np.sqrt(numerador/denominador)        
        
        Significances['Betard_0'][f'{file_name}_{channel}'][i,j] = Significances['Betard_minus1'][f'{file_name}_{channel}'][M][g_U]*factor

Significances['Betard_0'][f'{file_name}_{channel}'] = pd.DataFrame(Significances['Betard_0'][f'{file_name}_{channel}'])
Significances['Betard_0'][f'{file_name}_{channel}'].columns = [float(M) for M in Masses]
Significances['Betard_0'][f'{file_name}_{channel}'].index = [float(fila) for fila in g_Us]

Significances['Betard_0'][f'{file_name}_{channel}'].to_excel(f'{Path_significances}/Excel_Files/Betard_0/{file_name}_{channel}.xlsx')

In [ ]:
file_name = 'all'
channel = 'combined'

channel_1 = 'hadronic'
channel_2 = 'semileptonic'

process_1 = 'b_tau_tau' #sLQ
process_2 = 'b_b_tau_tau' #dLQ

Significances['Betard_0'][f'{file_name}_{channel}'] = np.zeros([len(g_Us), len(Masses)])

for i in range(len(g_Us)):
    for j in range(len(Masses)):
        g_U = float(g_Us[i])
        M = float(Masses[j])
        
        #sLQ_hadronic
        Eficiencia_M_wo_RHC_1 = Eficiencia(process_1, channel_1, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_1 = Eficiencia(process_1, channel_1, f'Tau_LQ_{Masses[j]}')
        Composition_wo_RHC_1 = Compositions['Betard_0'][f'{process_1}_{channel_1}'][M][g_U]
        Composition_w_RHC_1 = Compositions['Betard_minus1'][f'{process_1}_{channel_1}'][M][g_U]
        
        #sLQ_semiletpnic
        Eficiencia_M_wo_RHC_2 = Eficiencia(process_1, channel_2, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_2 = Eficiencia(process_1, channel_2, f'Tau_LQ_{Masses[j]}')
        Composition_wo_RHC_2 = Compositions['Betard_0'][f'{process_1}_{channel_2}'][M][g_U]
        Composition_w_RHC_2 = Compositions['Betard_minus1'][f'{process_1}_{channel_2}'][M][g_U]
        
        #dLQ_hadronic
        Eficiencia_M_wo_RHC_3 = Eficiencia(process_2, channel_1, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_3 = Eficiencia(process_2, channel_1, f'Tau_LQ_{Masses[j]}')
        Composition_wo_RHC_3 = Compositions['Betard_0'][f'{process_2}_{channel_1}'][M][g_U]
        Composition_w_RHC_3 = Compositions['Betard_minus1'][f'{process_2}_{channel_1}'][M][g_U]
        
        #dLQ_semiletpnic
        Eficiencia_M_wo_RHC_4 = Eficiencia(process_2, channel_2, f'Tau_LQ_{Masses[j]}_wo_RHC')
        Eficiencia_M_w_RHC_4 = Eficiencia(process_2, channel_2, f'Tau_LQ_{Masses[j]}')
        Composition_wo_RHC_4 = Compositions['Betard_0'][f'{process_2}_{channel_2}'][M][g_U]
        Composition_w_RHC_4 = Compositions['Betard_minus1'][f'{process_2}_{channel_2}'][M][g_U]
        
        
        XS_wo_RHC_sLQ = Tablas_XS['Betard_0']['sLQ'][M][g_U]
        XS_w_RHC_sLQ = Tablas_XS['Betard_minus1']['sLQ'][M][g_U]    
        
        numerador = (Eficiencia_M_wo_RHC_1/Composition_wo_RHC_1 + Eficiencia_M_wo_RHC_2/Composition_wo_RHC_2)*XS_wo_RHC_sLQ
        + (Eficiencia_M_wo_RHC_3/Composition_wo_RHC_3 + Eficiencia_M_wo_RHC_4/Composition_wo_RHC_4)*XS_wo_RHC_sLQ
        
        denominador = (Eficiencia_M_w_RHC_1/Composition_w_RHC_1 + Eficiencia_M_w_RHC_2/Composition_w_RHC_2)*XS_w_RHC_sLQ
        + (Eficiencia_M_w_RHC_3/Composition_w_RHC_3 + Eficiencia_M_w_RHC_4/Composition_w_RHC_4)*XS_w_RHC_sLQ
        
        factor = np.sqrt(numerador/denominador)        
        
        Significances['Betard_0'][f'{file_name}_{channel}'][i,j] = Significances['Betard_minus1'][f'{file_name}_{channel}'][M][g_U]*factor

Significances['Betard_0'][f'{file_name}_{channel}'] = pd.DataFrame(Significances['Betard_0'][f'{file_name}_{channel}'])
Significances['Betard_0'][f'{file_name}_{channel}'].columns = [float(M) for M in Masses]
Significances['Betard_0'][f'{file_name}_{channel}'].index = [float(fila) for fila in g_Us]

Significances['Betard_0'][f'{file_name}_{channel}'].to_excel(f'{Path_significances}/Excel_Files/Betard_0/{file_name}_{channel}.xlsx')